In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
data = pd.read_csv("MSFT.csv")

In [4]:
data.head()

,Date,Close Price,Open Price,High Price,Low Price,Total Traded Quantity
0,"Oct 24, 2019",139.94,139.14,140.40,138.69,37.28M
1,"Oct 23, 2019",137.24,136.88,137.45,135.61,31.38M
2,"Oct 22, 2019",136.37,138.97,140.01,136.26,27.94M
3,"Oct 21, 2019",138.43,138.45,138.50,137.01,20.72M
4,"Oct 18, 2019",137.41,139.76,140.00,136.56,32.28M


In [5]:
data.tail()

,Date,Close Price,Open Price,High Price,Low Price,Total Traded Quantity
498,"Oct 31, 2017",83.18,84.36,84.36,83.11,27.09M
499,"Oct 30, 2017",83.89,83.70,84.33,83.11,31.76M
500,"Oct 27, 2017",83.81,84.37,86.20,83.61,71.07M
501,"Oct 26, 2017",78.76,79.20,79.42,78.75,32.12M
502,"Oct 25, 2017",78.63,78.58,79.10,78.01,20.41M


In [6]:
data.describe()

,Close Price,Open Price,High Price,Low Price
count,503.000000,503.000000,503.000000,503.000000
mean,109.665805,109.707575,110.626839,108.608628
std,17.381956,17.414436,17.427872,17.310675
min,78.630000,78.580000,79.100000,78.010000
25%,95.385000,95.795000,96.505000,94.125000
50%,107.220000,107.460000,108.300000,105.960000
75%,124.245000,124.105000,125.470000,123.445000
max,141.570000,141.500000,142.370000,140.300000


In [7]:
data['Date'] = pd.to_datetime(data.Date)
data.sort_values(by = "Date", ascending = False, inplace = True)

In [8]:
data.head()

,Date,Close Price,Open Price,High Price,Low Price,Total Traded Quantity
0,2019-10-24,139.94,139.14,140.40,138.69,37.28M
1,2019-10-23,137.24,136.88,137.45,135.61,31.38M
2,2019-10-22,136.37,138.97,140.01,136.26,27.94M
3,2019-10-21,138.43,138.45,138.50,137.01,20.72M
4,2019-10-18,137.41,139.76,140.00,136.56,32.28M


In [9]:
max_close_price = np.max(data["Close Price"].iloc[:90])
print("Max_close_price: {}".format(max_close_price))
min_close_price = np.min(data["Close Price"].iloc[:90])
print("Min_close_price: {}".format(min_close_price))
mean_close_price = np.mean(data["Close Price"].iloc[:90])
print("Mean_close_price: {}".format(mean_close_price))

Max_close_price: 141.57
Min_close_price: 132.21
Mean_close_price: 137.447222222


In [10]:
data.columns

Index([u'Date', u'Close Price', u'Open Price', u'High Price', u'Low Price',
       u'Total Traded Quantity'],
      dtype='object')

In [11]:
data.dtypes

Date                     datetime64[ns]
Close Price                     float64
Open Price                      float64
High Price                      float64
Low Price                       float64
Total Traded Quantity            object
dtype: object

In [12]:
np.max(data["Date"]) - np.min(data["Date"])

Timedelta('729 days 00:00:00')

In [13]:
data["Month"] = data["Date"].dt.month
data["Year"] = data["Date"].dt.year

In [14]:
data.head()

,Date,Close Price,Open Price,High Price,Low Price,Total Traded Quantity,Month,Year
0,2019-10-24,139.94,139.14,140.40,138.69,37.28M,10,2019
1,2019-10-23,137.24,136.88,137.45,135.61,31.38M,10,2019
2,2019-10-22,136.37,138.97,140.01,136.26,27.94M,10,2019
3,2019-10-21,138.43,138.45,138.50,137.01,20.72M,10,2019
4,2019-10-18,137.41,139.76,140.00,136.56,32.28M,10,2019


In [15]:
data["Volume"] = data["Total Traded Quantity"].apply(lambda x: float(x[:-2]))

In [16]:
def vwap(df):
    q = df["Volume"].values
    p = df["Close Price"].values
    return df.assign(VWAP=(p * q).cumsum() / q.cumsum())

data = data.groupby(["Year", "Month"], group_keys=False).apply(vwap)

In [17]:
def avg_price_n(n):
    df  = data[:n]
    return np.average(df["Close Price"])

In [18]:
def profit_loss_n(n):
    return ((data["Close Price"][0] - data["Close Price"][n-1])/data["Close Price"][n-1])*100

In [19]:
profit_loss_n(1)

0.0

In [20]:
print(avg_price_n(7))
print(avg_price_n(2*7))
print(avg_price_n(30))
print(avg_price_n(90))
print(avg_price_n(180))
print(avg_price_n(365))

82.25428571428571
82.82500000000002
83.62366666666667
87.25544444444445
92.27077777777778
100.73956164383563


In [21]:
print(profit_loss_n(7))
print(profit_loss_n(14))
print(profit_loss_n(30))
print(profit_loss_n(90))
print(profit_loss_n(180))
print(profit_loss_n(365))

-0.33473399330531933
2.056592765460905
1.9079522283716899
3.132139435477928
32.43115359136935
43.79367036580354


In [22]:
data["Day_Perc_Change"] = data["Close Price"].pct_change()
data = data.drop([498], axis=0)

In [23]:
def trend(x):
    if x>=-0.5 and x<0.5:
         return "Slight or No Change"
    elif x>=0.5 and x<1:
        return "Slight positive"
    elif x>=1 and x<3:
        return "positive"
    elif x>=-1 and x<-0.5:
        return "Slight negative"
    elif x>=-3 and x<-1:
        return "negative"
    elif x>3 and x<7:
        return "Top gainers"
    elif x<-3 and x>=-7:
        return "Top losers"
    elif x>=7:
        return "Bull run"
    elif x<-7:
        return "Bear drop"

In [24]:
data["Trend"] = data["Day_Perc_Change"].apply(trend)

In [25]:
def avg_trend(df):
    vol = df["Volume"].values
    avg = np.average(vol)
    return df.assign(Avg_Trend=avg)

def median_trend(df):
    vol = df["Volume"].values
    med = np.median(vol)
    return df.assign(Median_Trend=med)

In [26]:
data.groupby(["Trend"], group_keys=False).apply(avg_trend)
data = data.groupby(["Trend"], group_keys=False).apply(median_trend)

In [31]:
data.tail()

,Date,Close Price,Open Price,High Price,Low Price,Total Traded Quantity,Month,Year,Volume,VWAP,Day_Perc_Change,Trend,Avg_Trend,Median_Trend
13,2019-10-07,137.12,137.14,138.18,137.02,16.60M,10,2019,16.6,138.490646,0.010688,Slight or No Change,28.28247,25.7
14,2019-10-04,138.12,136.75,138.25,136.42,23.84M,10,2019,23.8,138.465915,0.007293,Slight or No Change,28.28247,25.7
15,2019-10-03,136.28,134.95,136.75,133.22,26.38M,10,2019,26.3,138.315812,-0.013322,Slight or No Change,28.28247,25.7
16,2019-10-02,134.65,136.25,136.37,133.58,32.00M,10,2019,32.0,138.033147,-0.011961,Slight or No Change,28.28247,25.7
17,2019-10-01,137.07,139.66,140.25,137.00,22.58M,10,2019,22.5,137.983614,0.017973,Slight or No Change,28.28247,25.7


In [29]:
data.to_csv("week2.csv", index=False)